In [ ]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
#import networkx as nx
#from networkx.algorithms.community.label_propagation import label_propagation_communities
import networkx.drawing.nx_pylab

In [ ]:
#!pip install -U git+https://github.com/taynaud/python-louvain.git@networkx2
#from community import community_louvain

In [2]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)

In [8]:
# Build the table-column details files

bi_tables_df = pd.read_parquet("data/BI.table-col-details.parquet")
BASE_DIR = "C:\\obsidian\\datacatalog\\BI\\_Tables"
for row in bi_tables_df.itertuples():
    Path(f"{BASE_DIR}\\{row.schema_name}").mkdir(parents=True, exist_ok=True)
    if os.path.exists(f"{BASE_DIR}\\{row.schema_name}\\{row.table_name}.md"):
        with open(f"{BASE_DIR}\\{row.schema_name}\\{row.table_name}.md", 'a') as f:
            f.write(f"|  {row.column_name} | {row.data_type_ext} | {row.nullable} | {row.default_value} | {row.primary_key} |\n")
    else:
        with open(f"{BASE_DIR}\\{row.schema_name}\\{row.table_name}.md", 'a') as f:
            f.write(f"**Table Name: {row.table_name}** \n\n")
            f.write(f"| Column_Name | Data_Type | Nullable | Default | IS PK |\n")
            f.write(f"| --- | --- | --- | --- | --- |\n")
            f.write(f" | {row.column_name} | {row.data_type_ext} | {row.nullable} | {row.default_value} | {row.primary_key} |\n")



In [37]:
# Build the view-column details files

bi_views_df = pd.read_parquet("data/BI.view-col-details.parquet")
BASE_DIR = "C:\\obsidian\\datacatalog\\BI\\Views"
for row in bi_views_df.itertuples():
    Path(f"{BASE_DIR}\\{row.schema_name}").mkdir(parents=True, exist_ok=True)
    if os.path.exists(f"{BASE_DIR}\\{row.schema_name}\\{row.view_name}.md"):
        with open(f"{BASE_DIR}\\{row.schema_name}\\{row.view_name}.md", 'a') as f:
            f.write(f"|  {row.column_name} | {row.data_type_ext} | {row.nullable} | {row.comments} |\n")
    else:
        with open(f"{BASE_DIR}\\{row.schema_name}\\{row.view_name}.md", 'a') as f:
            f.write(f"**View Name: {row.view_name}** \n\n")
            f.write(f"| Column_Name | Data_Type | Nullable | Comments |\n")
            f.write(f"| --- | --- | --- | --- |\n")
            f.write(f"| {row.column_name} | {row.data_type_ext} | {row.nullable} | {row.comments} |\n")



In [33]:
bi_views_df = pd.read_parquet("data/BI.views.parquet")
bi_views_df

,name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,is_published,is_schema_published,lob_data_space_id,filestream_data_space_id,max_column_id_used,lock_on_bulk_load,uses_ansi_nulls,is_replicated,has_replication_filter,is_merge_published,is_sync_tran_subscribed,has_unchecked_assembly_data,text_in_row_limit,large_value_types_out_of_row,is_tracked_by_cdc,lock_escalation,lock_escalation_desc,is_filetable,is_memory_optimized,durability,durability_desc
0,AX_Dimension_Structures,5575058,None,1,0,U,USER_TABLE,2015-06-09 08:41:56.743,2022-05-28 01:36:08.937,False,False,False,0,None,8,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
1,iTrak_DPSSafety_List,6291082,None,1,0,U,USER_TABLE,2018-02-23 09:53:16.613,2018-08-01 15:24:11.800,False,False,False,1,None,22,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
2,MSDGen_DangerousGoods,13243102,None,1,0,U,USER_TABLE,2017-08-15 15:40:50.580,2017-08-15 15:40:50.583,False,False,False,0,None,10,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
3,zPRDSC_SP_NewVendor_RequestsSharepoint,13959126,None,1,0,U,USER_TABLE,2019-01-17 13:30:04.477,2019-08-14 10:58:38.267,False,False,False,1,None,62,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
4,Husky_Fuel_Trans,14623095,None,1,0,U,USER_TABLE,2016-06-16 12:03:40.063,2016-06-16 12:03:40.123,False,False,False,0,None,32,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,PetroTranz_FormA,2131538677,None,1,0,U,USER_TABLE,2019-06-03 14:10:45.120,2019-06-04 12:22:43.743,False,False,False,1,None,67,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
312,BI_Calendar,2132202646,None,1,0,U,USER_TABLE,2017-04-24 12:14:23.117,2017-04-24 12:25:43.647,False,False,False,0,None,13,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
313,SCADA_TagValues2,2136394680,None,1,0,U,USER_TABLE,2018-11-06 10:35:42.203,2018-11-06 10:35:42.203,False,False,False,0,None,6,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
314,MSDGen_ProductList,2144726693,None,1,0,U,USER_TABLE,2017-08-10 13:25:58.530,2017-08-30 11:40:29.453,False,False,False,0,None,50,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA


In [34]:
bi_tables_df = pd.read_parquet("data/BI.tables.parquet")
bi_tables_df

,name,object_id,principal_id,schema_id,parent_object_id,type,type_desc,create_date,modify_date,is_ms_shipped,is_published,is_schema_published,lob_data_space_id,filestream_data_space_id,max_column_id_used,lock_on_bulk_load,uses_ansi_nulls,is_replicated,has_replication_filter,is_merge_published,is_sync_tran_subscribed,has_unchecked_assembly_data,text_in_row_limit,large_value_types_out_of_row,is_tracked_by_cdc,lock_escalation,lock_escalation_desc,is_filetable,is_memory_optimized,durability,durability_desc
0,AX_Dimension_Structures,5575058,None,1,0,U,USER_TABLE,2015-06-09 08:41:56.743,2022-05-28 01:36:08.937,False,False,False,0,None,8,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
1,iTrak_DPSSafety_List,6291082,None,1,0,U,USER_TABLE,2018-02-23 09:53:16.613,2018-08-01 15:24:11.800,False,False,False,1,None,22,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
2,MSDGen_DangerousGoods,13243102,None,1,0,U,USER_TABLE,2017-08-15 15:40:50.580,2017-08-15 15:40:50.583,False,False,False,0,None,10,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
3,zPRDSC_SP_NewVendor_RequestsSharepoint,13959126,None,1,0,U,USER_TABLE,2019-01-17 13:30:04.477,2019-08-14 10:58:38.267,False,False,False,1,None,62,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
4,Husky_Fuel_Trans,14623095,None,1,0,U,USER_TABLE,2016-06-16 12:03:40.063,2016-06-16 12:03:40.123,False,False,False,0,None,32,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,PetroTranz_FormA,2131538677,None,1,0,U,USER_TABLE,2019-06-03 14:10:45.120,2019-06-04 12:22:43.743,False,False,False,1,None,67,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
312,BI_Calendar,2132202646,None,1,0,U,USER_TABLE,2017-04-24 12:14:23.117,2017-04-24 12:25:43.647,False,False,False,0,None,13,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
313,SCADA_TagValues2,2136394680,None,1,0,U,USER_TABLE,2018-11-06 10:35:42.203,2018-11-06 10:35:42.203,False,False,False,0,None,6,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA
314,MSDGen_ProductList,2144726693,None,1,0,U,USER_TABLE,2017-08-10 13:25:58.530,2017-08-30 11:40:29.453,False,False,False,0,None,50,False,True,False,False,False,False,False,0,False,False,0,TABLE,False,False,0,SCHEMA_AND_DATA


In [35]:
bi_refs_df = pd.read_parquet("data/BI.refs.parquet")
bi_refs_df

,referencing_schema_name,referencing_obj_name,referencing_id,referencing_minor_id,referencing_class,referencing_class_desc,is_schema_bound_reference,referenced_class,referenced_class_desc,referenced_server_name,referenced_database_name,referenced_schema_name,referenced_entity_name,referenced_id,referenced_minor_id,is_caller_dependent,is_ambiguous
0,dbo,AX_FixedAsset_TransDetails,4195065,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,AX_SECURE_PRD_REPLICA,dbo,ASSETTABLE,NaN,0,False,False
1,dbo,AX_FixedAsset_TransDetails,4195065,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,AX_SECURE_PRD_REPLICA,dbo,ASSETTRANS,NaN,0,False,False
2,dbo,AX_FixedAsset_TransDetails,4195065,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,None,dbo,AX_Default_Dimensions,1.655756e+08,0,False,False
3,dbo,zExcel_DPS_CustomerRevenue,25767149,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,BI,dbo,AX_default_dimensions,1.655756e+08,0,False,False
4,dbo,zExcel_DPS_CustomerRevenue,25767149,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,BI,dbo,AX_GL_TransDetails,4.215765e+08,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,dbo,AX_Vendor_Trans,2135678656,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,AX_SECURE_PRD_REPLICA,dbo,VENDDIRPARTYTABLEVIEW,NaN,0,False,False
1459,dbo,AX_Vendor_Trans,2135678656,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,AX_SECURE_PRD_REPLICA,dbo,VENDTABLE,NaN,0,False,False
1460,dbo,AX_Vendor_Trans,2135678656,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,None,AX_SECURE_PRD_REPLICA,dbo,VENDTRANS,NaN,0,False,False
1461,dbo,PRD_SC_VendorRequests,2140586714,0,1,OBJECT_OR_COLUMN,False,1,OBJECT_OR_COLUMN,Azure-PROD,APPS,dbo,PRD_SC_VendorRequests,NaN,0,False,False


In [14]:
G = nx.Graph()

In [15]:
#G.add_nodes_from(new_df['parent'])
#G.add_nodes_from(new_df['child'])
#G.nodes

In [16]:
for edge in new_df.itertuples():
    G.add_edge(edge.parent, edge.child)

NameError: name 'new_df' is not defined

In [ ]:
plt.figure(figsize=(10, 10))
nx.draw(G, with_labels=False)

In [ ]:
weights = [1 if G[u][v] == {} else G[u][v]['weight'] for u,v in G.edges()]
plt.figure(figsize=(10, 10))
nx.draw(G, with_labels=False, width=weights)


In [ ]:
pos = nx.circular_layout(G)
plt.figure(figsize=(10, 10))
nx.draw(G, pos=pos, with_labels=False)


In [ ]:
print(sorted(G.degree, key=lambda x: x[1], reverse=True))

In [ ]:
communities = label_propagation_communities(G)
print([community for community in communities])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

def community_layout(g, partition):
    """
    Compute the layout for a modular graph.


    Arguments:
    ----------
    g -- networkx.Graph or networkx.DiGraph instance
        graph to plot

    partition -- dict mapping int node -> int community
        graph partitions


    Returns:
    --------
    pos -- dict mapping int node -> (float x, float y)
        node positions

    """

    pos_communities = _position_communities(g, partition, scale=3.)

    pos_nodes = _position_nodes(g, partition, scale=1.)

    # combine positions
    pos = dict()
    for node in g.nodes():
        pos[node] = pos_communities[node] + pos_nodes[node]

    return pos

def _position_communities(g, partition, **kwargs):

    # create a weighted graph, in which each node corresponds to a community,
    # and each edge weight to the number of edges between communities
    between_community_edges = _find_between_community_edges(g, partition)

    communities = set(partition.values())
    hypergraph = nx.DiGraph()
    hypergraph.add_nodes_from(communities)
    for (ci, cj), edges in between_community_edges.items():
        hypergraph.add_edge(ci, cj, weight=len(edges))

    # find layout for communities
    pos_communities = nx.spring_layout(hypergraph, **kwargs)

    # set node positions to position of community
    pos = dict()
    for node, community in partition.items():
        pos[node] = pos_communities[community]

    return pos

def _find_between_community_edges(g, partition):

    edges = dict()

    for (ni, nj) in g.edges():
        ci = partition[ni]
        cj = partition[nj]

        if ci != cj:
            try:
                edges[(ci, cj)] += [(ni, nj)]
            except KeyError:
                edges[(ci, cj)] = [(ni, nj)]

    return edges

def _position_nodes(g, partition, **kwargs):
    """
    Positions nodes within communities.
    """

    communities = dict()
    for node, community in partition.items():
        try:
            communities[community] += [node]
        except KeyError:
            communities[community] = [node]

    pos = dict()
    for ci, nodes in communities.items():
        subgraph = g.subgraph(nodes)
        pos_subgraph = nx.spring_layout(subgraph, **kwargs)
        pos.update(pos_subgraph)

    return pos

def test():
    # to install networkx 2.0 compatible version of python-louvain use:
    # pip install -U git+https://github.com/taynaud/python-louvain.git@networkx2
    from community import community_louvain

    g = nx.karate_club_graph()
    partition = community_louvain.best_partition(g)
    pos = community_layout(g, partition)

    nx.draw(g, pos, with_labels=True, node_color=list(partition.values())); plt.show()
    return


In [ ]:
plt.figure(figsize=(100, 100))

partition = community_louvain.best_partition(G)
pos = community_layout(G, partition)
weights = [1 if G[u][v] == {} else G[u][v]['weight'] for u,v in G.edges()]
nx.drawing.nx_pylab.draw_kamada_kawai(G)
#nx.draw(G, pos, with_labels=True, width=weights, node_color=list(partition.values()));
plt.show()

In [ ]:
plt.figure(figsize=(100, 100))
#nx.draw(G, with_labels=True, node_size=100, node_color="skyblue", pos=nx.fruchterman_reingold_layout(G))
#nx.draw(G, with_labels=True, node_size=100, node_color="skyblue", pos=nx.spectral_layout(G))
nx.draw(G, with_labels=True, node_size=100, node_color="skyblue", pos=nx.circular_layout(G))
plt.title("BI31")
plt.show()
